# Weight timeseries data analysis
Here we will analyze a (fake) dataset of the weight of twenty individuals over time. The data is split up into two files:

1. `weight_data.csv` with columns: `|| person | Day_1 | ... | Day_6 ||`
2. `weight_metadata.csv`: `|| person | sex | homestate | ... | feature_n ||`

The challenges involved with this assignment will include transforming, merging, and aggregating this data in different ways in order to visualize different aspects. This assignment is for someone who has had at least some exposure to `pandas` and plotting with `seaborn`.

If you're interested in seeing how this data was generated, take a look [here](https://github.com/michaelsilverstein/Pandas-and-Plotting/blob/master/weight_timeseries/weight_timeseries_generate_data.ipynb)

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Brief Seaborn introduction
- Longform data
- `hue`

# Timeseries
- show how to apply a string split to a whole column of a longform dataframe
- melt
## Timeseries + metadata
- merge

# Timeseries $\Delta$Baseline

# Comparing gender
- take mean in order to ensure independence

# Scatter with calorie intake?
- Ask for matplotlib